In [2]:
import pandas as pd
import json
import time
import os

In [ ]:
tc1 = pd.read_csv('medicamentos_original.csv', dtype=object)
cnpjs = tc1['CNPJ'].to_list()
cnpjs = pd.DataFrame(cnpjs, columns=['CNPJ'])
cnpjs['CNPJ'] = cnpjs['CNPJ'].apply(lambda x: x.replace("-", "").replace(".", "").replace("/", ""))

In [ ]:
cnpjs = cnpjs.drop_duplicates()
fix_cnpjs = []
for v in cnpjs['CNPJ']:
	if len(v.strip()) != 14:
		vs = v.strip()
		fix_cnpjs.append(f'{vs:0>14}')
	else:
		fix_cnpjs.append(v.strip())
cnpjs['CNPJ'] = fix_cnpjs
cnpjs.to_csv('cnpjs.csv', index=False)

In [60]:
df = pd.read_csv('cnpjs.csv', dtype=str)

## Preprocessing

In [5]:
!pip install grequests
import grequests
import sys
del sys.modules["gevent.monkey"]
import grequests
import requests

Defaulting to user installation because normal site-packages is not writeable


In [4]:
def error_handler(request, exception):
  print("Exception:", exception)
  
def renew_cookies(old_cookies):
    main = requests.get("https://consultas.anvisa.gov.br/#/medicamentos/")
    with open("main.html", 'w') as f:
        f.write(main.text)
    new_cookies = dict(main.headers)["Set-Cookie"].split(";")[:3]
    if "FGTServer" in dict(main.headers)["Set-Cookie"]:
        new_cookies[1] = new_cookies[2].split(", ")[1]
        cookies = ";".join(new_cookies[:2])
    else:
        cookies = [old_cookies.split(";")[0]]
        cookies.append(new_cookies[0])
        cookies = ";".join(cookies)

    return cookies

In [11]:
# Reset stored cookies
# https://consultas.anvisa.gov.br/#/medicamentos/
cookies = ""

In [66]:
step = 1
total = len(df)
idx = 0
for curr in range(idx + step, total + 1, step):
	try:
		print(f"\rLoading {curr}/{total}", end='')
		# Renew header
		cookies = renew_cookies(cookies)
		header = {
              "Host": "consultas.anvisa.gov.br",
              "Referer": "https://consultas.anvisa.gov.br/",
              "Authorization": "Guest",
              "Cookie": cookies,
              "Connection": "keep-alive",
              "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
              }
		basic_requests = [grequests.get(f"https://consultas.anvisa.gov.br/api/consulta/medicamento/produtos/?count=1000&filter[cnpj]={row['CNPJ']}&page=1",
                    headers=header) for _, row in df[idx:curr].iterrows()]
    
		responses = grequests.map(basic_requests, exception_handler=error_handler)

		advanced_requests = []
		i = 0
		names_cnpj = []
		for response in responses:
			with open(f"cnpjs/{df['CNPJ'][idx + i]}.json", 'w') as f:
				f.write(response.text)
			i+=1
		time.sleep(2)
	except Exception as e:
		print("Outside error", e)
	finally:
		idx += step


Loading 428/427

In [68]:
data = {
	'codigo_anvisa': [],
	'registro': [],
	'nome': [],
	'cnpj': [],
	'razao_social': []
}
for cnpjfile in os.listdir('cnpjs'):
	with open(f'cnpjs/{cnpjfile}', 'r') as f:
		results = json.load(f)
		for result in results['content']:
			data['codigo_anvisa'].append(result['produto']['codigo'])
			data['registro'].append(result['produto']['numeroRegistro'])
			data['nome'].append(result['produto']['nome'])
			data['cnpj'].append(result['empresa']['cnpj'])
			data['razao_social'].append(result['empresa']['razaoSocial'])
df = pd.DataFrame(data)
df.to_csv('medicamentos_auto.csv', index=False)

In [10]:
import random
df = pd.read_csv('medicamentos_auto.csv')
for n, row in df.iterrows():
	if os.path.exists(f"medicamentos/{row['registro']}.json"):
		continue
	print(f"\rLoading {n}/{len(df)}", end='')
	codigo = row['codigo_anvisa']
	cookies = renew_cookies(cookies) 
	advanced_requests = []
	header = {
              "Host": "consultas.anvisa.gov.br",
              "Referer": "https://consultas.anvisa.gov.br/",
              "Authorization": "Guest",
              "Cookie": cookies,
              "Connection": "keep-alive",
              "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36"
              }
	advanced_requests.append(grequests.get(f"https://consultas.anvisa.gov.br/api/consulta/medicamento/produtos/codigo/{codigo}",
									headers=header))
	responses = grequests.map(advanced_requests, exception_handler=error_handler)
	for response in responses:
		with open(f"medicamentos/{row['registro']}.json", 'w') as f:
			f.write(response.text)
	time.sleep(random.randint(1, 2))

Loading 12967/27354

KeyboardInterrupt
2024-06-27T02:46:59Z


KeyboardInterrupt: 